## Functions
#### types of fucntions
    - operators and primitive functions
    - q-sql functions (i.e. select, update, upsert, insert, etc.)
    - user defined functions
    - adverbs or derived functions

In [1]:
// Infix and prefix notations
show 5 + 7 // infix notation
show +[5;7] // prefix notation
show (+) . 5 7 // apply notation

12
12
12


A q function is deliminated with {} and has optional [] for the declared variables
x,y,z are implicitly defined as inputs

In [2]:
{x*x} 5f

25f


In [3]:
{[int] int*int} 5f

25f


- prefix notation [] maybe needed when using longer expressions due to right to left evaluation
- Also good for code clerity
- Functions can ne nested

In [4]:
a:{[x;w]v:{[x;w] 3*w+x};v[x;2*w]-4f}
a[4f;3f]

26f


Functions and variables defined within the fucntion are local to the functions.

To declare global inside the function use ::

In [4]:
// v has no visibility from above
v

[0;31mv[0m: [0;31mv[0m

In [5]:
f:{[b]a::6;a+b}
show f[7];
a // global

13


6


Multivariate functions can be projected by specifying only part of the input params.

Projecton is related to currying in functional programming.

Monadic function can be a projection of dyadic funciton by fixing all except one input param

In [6]:
f:{x+y}
g:f[4;]
g[5]

9


### 3.1Namespace
A namespace is a context with which we resolve problems of unique naming. You can define variables and functions inside it.
It can be used to resolve local vs global variable avilability.

#### Limits in kdb/q:
- Input params for function (8 max)
- Constants (max 96)
- Global variables (32 max)
- Locals variables (24 max)
- More than 999 nested columns in splayed table

In [7]:
.namespace1.x:4f;
.namespace1.f:{5+x};
.namespace2.x:5;
.namespace2.f:{1f+log x};
// we can call from both
.namespace2.f[.namespace1.x]

2.386294


#### 3.1.01 .quantQ Namespace
In this book all the fucntions and variables defined are in this .quantQ namespace. All relavant funcitons are furthur listed in sub-namespaces (namespace with in the namespace) in logocal grouping.

In [8]:
\l quantQ-master\lib\quantQ_complex.q 
// Note: trying to load in the entire quantQ.q namespace results in error (Ram issue?)

In [9]:
// subnamespace complex for fourier transformation 
\f .quantQ.complex

`conjugate`dft`dft1k`dftTab`dftTs`div`exp`fractal`log`mandelbrot`mandelbrot_`..


### 3.2 The six adverbs
 - Kdb has nouns, verbs and adverbs
 - Nouns : variables and functions are nouns
 - syntactic structures and arectmatic and logical operators are verbs like {} [] or + -
 - Adverbs are higer order functions that modify the applied to lists. They return derived functions known as derivatives.

##### Adverbs
- 1) each
   -  1.2) each left \:
   -  1.3) each right /:
   -  1.4) cross product (/:\: (combination of each left and each right) or cross)
   -  1.5) each both '
   -  1.6) each prior ':
- 2) compose (')
- 3) Over/Fold /
- 4) Scan \
- 5) converge / \
- 6) cross product `/:\:` (or `cross` for cartisian product) 

In [10]:
// each modifies a monadic function, i.e. reverse
reverse each ((`a`b`c);(`d`e`f))

c b a
f e d


In [11]:
// without each
reverse((`a`b`c);(`d`e`f))

d e f
a b c


In [12]:
// Each left modifies a dyadic function to apply on each item in the left
("Hello ";"Greetings ";"Welcome "),\: "Mo"

"Hello Mo"
"Greetings Mo"
"Welcome Mo"


In [13]:
// Each right modifies a dyadic function to apply on each item in the right
"Hello ",/:("Alice";"Bob";"Mo")

"Hello Alice"
"Hello Bob"
"Hello Mo"


In [14]:
// usecase in creating an identity matrix
.quantQ.identity:{[n] `float$(til n)=/:til n}
.quantQ.identity[5]

1 0 0 0 0
0 1 0 0 0
0 0 1 0 0
0 0 0 1 0
0 0 0 0 1


In [15]:
// cross product or Cartesian product is the composition of an each right withy an each left
raze (4 5 6f) ,/:\:(5 6 7f)

4 5
4 6
4 7
5 5
5 6
5 7
6 5
6 6
6 7


In [16]:
(4 5 6f) cross (5 6 7f)

4 5
4 6
4 7
5 5
5 6
5 7
6 5
6 6
6 7


In [17]:
// cross is set, but /:\: is more versitile and can modify a monadic function
(4 5 6f) */:\:(5 6 7f)

20 24 28
25 30 35
30 36 42


In [18]:
// simmilar to dot notation
{x*y} . ((4 5 6f);(5 6 7f))

20 30 42f


In [19]:
// Each both, apllied pairwise to items on lists (of the same size)
show (4 5 6f) *'(5 6 7f)

20 30 42f


In [19]:
// returns 'length error when it's not the same `cardinality` (size)
(1 2 3) +'(4 5 6 7)

[0;31mlength[0m: [0;31mlength[0m

In [20]:
// good usecase of each both is to add columm to table
tbl:([sym:20?`A`B`C`D] market:20?`US`UK`CH`FR;price:20?500.00);
show tbl1:select symCount:count i by sym from tbl where market=`US;
show tbl2:select avgPrice:avg price by sym from tbl where market=`US;

(0!tbl1) ,' select avgPrice from tbl2 // pairwise simple join

sym| symCount
---| --------
A  | 2       
B  | 2       
D  | 1       
sym| avgPrice
---| --------
A  | 217.5092
B  | 180.7113
D  | 289.2601


sym symCount avgPrice
---------------------
A   2        217.5092
B   2        180.7113
D   1        289.2601


In [21]:
//note in many instances q's build in function is sufficent
(sum 1 2 3f * ' 3 4 5f) ~ sum 1 2 3f * 3 4 5f

1b


In [22]:
// $ - built in efficent dot product
(1 2 3f)$(3 4 5f)

26f


Here we show a more complex example of each both. It'll use a the calc function which takes in a nested list and a scaler and applies it  to create a new column

In [23]:
n:100
t:([]a:-2.5 + (n#3) ?\: 5f; b:n?10)
t

a                                 b
-----------------------------------
-1.150396  2.228482    0.06312166 3
0.2298615  1.824631    -0.942818  3
1.566239   -1.854031   -1.761227  7
-1.128865  0.3175266   1.919115   8
-1.280403  0.8590627   1.819796   2
1.719904   0.2131853   -2.112133  1
0.6873184  2.380623    0.1984082  4
1.081429   -0.5568522  0.1114899  2
1.862513   0.7843671   2.312578   8
-0.6425137 -1.62767    0.4486008  0
2.275451   0.5792573   1.012275   5
-0.7248097 -0.6924283  2.044766   8
-1.468716  -0.09089507 -1.467188  5
0.1145891  -0.830597   -0.4268948 2
2.362907   0.2113628   0.5582909  8
-0.7925044 2.258373    -1.915262  6
1.579479   0.5457697   2.415397   9
1.271561   -0.5931603  -2.344368  0
-0.4026544 -1.568566   1.479765   0
-0.6663624 0.08261704  -2.172915  0
..


In [24]:

calc:{$[all 0 < x;x xexp reciprocal y;x xexp y]}

calc[t[`a][0];t[`b][0]]

-1.522446 11.06693 0.0002514984


In [25]:
update c: calc'[a;b] from t

a                                 b c                                     
--------------------------------------------------------------------------
-1.150396  2.228482    0.06312166 3 -1.522446  11.06693       0.0002514984
0.2298615  1.824631    -0.942818  3 0.01214504 6.074706       -0.8380763  
1.566239   -1.854031   -1.761227  7 23.12097   -75.30413      -52.56622   
-1.128865  0.3175266   1.919115   8 2.63716    0.0001033336   183.9958    
-1.280403  0.8590627   1.819796   2 1.639432   0.7379887      3.311656    
1.719904   0.2131853   -2.112133  1 1.719904   0.2131853      -2.112133   
0.6873184  2.380623    0.1984082  4 0.91052    1.242146       0.6674057   
1.081429   -0.5568522  0.1114899  2 1.169489   0.3100844      0.01243     
1.862513   0.7843671   2.312578   8 1.080843   0.9700965      1.110483    
-0.6425137 -1.62767    0.4486008  0 1          1              1           
2.275451   0.5792573   1.012275   5 1.178728   0.8965496      1.002443    
-0.7248097 -0.6924283  2.

What about if column b was also nested?

In [26]:
t:([]a:-2.5 + (n#3) ?\: 5f; b:-2.5 + (n#3) ?\: 5f)

In [27]:
t

a                                   b                                
---------------------------------------------------------------------
2.437455    1.535103    1.463833    -0.7663692  -1.479921  -1.291772 
1.597651    -1.266523   -0.989309   -1.314511   -1.720605  -0.3127593
-0.6890435  0.9980537   1.124177    1.345071    1.840338   0.7115395 
-0.6196833  1.034633    0.008489707 1.53274     -1.94559   -1.393202 
0.07059285  0.3875733   -0.3547036  -0.03615975 -1.297779  1.679566  
1.253362    1.562599    -0.9997442  1.40428     1.680652   2.41573   
2.317354    1.043307    0.3830546   -1.806206   2.178324   -1.202933 
0.9633372   -1.603524   0.4563688   -1.176237   -0.9048044 -0.5121039
-0.01880895 0.001495699 -0.01061332 1.810424    -0.976364  0.5334768 
1.539587    1.492969    -0.964727   1.871446    1.022177   -1.150008 
1.892622    1.458575    2.184048    0.6060803   1.899847   0.2931551 
2.177047    -1.900767   2.430673    -1.115314   -1.472695  1.459896  
-1.006319   2.422636

In [28]:
update c: calc'[a;b] from t

a                                   b                                 c      ..
-----------------------------------------------------------------------------..
2.437455    1.535103    1.463833    -0.7663692  -1.479921  -1.291772  0.31268..
1.597651    -1.266523   -0.989309   -1.314511   -1.720605  -0.3127593 0.54015..
-0.6890435  0.9980537   1.124177    1.345071    1.840338   0.7115395         ..
-0.6196833  1.034633    0.008489707 1.53274     -1.94559   -1.393202         ..
0.07059285  0.3875733   -0.3547036  -0.03615975 -1.297779  1.679566   1.10059..
1.253362    1.562599    -0.9997442  1.40428     1.680652   2.41573    1.37317..
2.317354    1.043307    0.3830546   -1.806206   2.178324   -1.202933  0.62794..
0.9633372   -1.603524   0.4563688   -1.176237   -0.9048044 -0.5121039 1.04491..
-0.01880895 0.001495699 -0.01061332 1.810424    -0.976364  0.5334768         ..
1.539587    1.492969    -0.964727   1.871446    1.022177   -1.150008  2.24242..
1.892622    1.458575    2.184048    0.60

#### Each Prior
a dyadic operatioon on each item of a list with it's predecessor

In [29]:
4f {show (x;y);x-y xexp 2}': 5 6 7f

5 4f
6 5f
7 6f


-11 -19 -29f


#### Compose(')
The composition of a multivariate function f[a,b,c,...] with a monadic function g[x], expressed as g(f) is expressed as '[g;f]

In [30]:
f:{[a;b;c;d] a+(2*b)+(3*c)+4*d}; // multivariate function
g:{log x}; // monadic function
d:('[g;f]); // applies the multivariate first then monadic
d[5;6;7;8]

4.248495


#### Over and Fold /
Over adverb / modified a dyadic function to accumulate the results over a list.
Let us denote $y_{i} , i \in \mathbb{N}^{*}_{+}$. The over adverb recursivly applies elements of y recursivly.

In [31]:
4f {x-y xexp 2}/(5 6 7f)

-106f


In [32]:
// equvalint to
(((4f-5f xexp 2)-6f xexp 2)-7f xexp 2)

-106f


In [33]:
// can also work without spliting the list
({x-y xexp 2}/) (4 5 6 7f)

-106f


The over adverb is called fold, when the function has more than 2 arguments

In [34]:
({z+x-y xexp 2}/)[1 3f;4 5f;2f]

-36 -34f


In [35]:
// equivalant to
(2f + (2f+1f-4f xexp 2)-5f xexp 2; 2f + (2f+3f-4f xexp 2)-5f xexp 2)

-36 -34f


The adverb fold can be thought of in a map-reduce context, where we fold (aka reduce) the result set.

#### Scan \

It's simmilar to the behavior of over, except it shows the intermediate results


In [36]:
({x-y xexp 2}\) (4 5 6 7f)

4 -21 -57 -106f


#### EMA: The Exponential Moving Average

Lets show how scan can be used to compuite this very common itterative function for which we want the intermedeate results.

Given a vector $x_{0}, ... x_{n}$ , and weight $\lambda$ , the EMA is:
   $EMA_{i} (\lambda) = x_{i-1} * (1-\lambda ) + EMA_{i-1} * \lambda$

In [37]:
\l quantQ-master\lib\quantQ_stats.q
.quantQ.stats.expma1

{[lambda;vector]
    / lambda -- memory
    / vector -- data
    {[x;y;z] (x*y)+z}\[ first vector; 1 - lambda; vector * lambda]
 }


In [38]:
.quantQ.stats.expma1[.2;10 20 30]

10 12 15.6


In [39]:
.quantQ.stats.expma

{[lambda;vector]
    // lambda -- memory
    // vector -- data
    :(first vector)(1-lambda)\vector*lambda;
 }


#### Converge
The over and scan adverbs can also be used to itterate over a list until convergence or until a loop is detected.

#### converge-repeate
applies the function f x times with initial value y. If x is a boolian function to indicate a condition on recursive iteration, the expression is equvalant to a while loop.

In [40]:
2 {x*x-1}/(2 4 3 2 5f)

2 132 30 2 380f


In [41]:
{x<20}{show x;x*x}/2

2
4
16


256


#### converge-iterate
The converge-itterate pattern is a simmilar to the converge-repete pattern describe in the previous section but using scan adverb to display all intermediary. Hence, the previous example applied using the scan \ adverb.

In [42]:
2 {x*x-1}\(2 4 3 2 5f)

2 4   3  2 5  
2 12  6  2 20 
2 132 30 2 380


In [43]:
{x<20}{show x;x*x}\2

2
4
16


2 4 16 256


### 3.3 Apply
##### @ (apply)
The verb @ is used to apply a multivariate mapping to a list L with the syntax: @[L;I;f]. The following expression applies the function cosine on the first and third elements of the list; the second element 1.570796 being left unchanged.

In [44]:
@[(0.7853982; 1.570796; 3.141592654);0 2;cos]

0.7071068 1.570796 -1


When applied to dyadic function. @[L;I;f;a] applies a dyadic function f on the I-th element of a list L with seconds arguments a.

In [45]:
@[1 2 3 4f; 0 2;{x+2*y};6 7f]

13 2 17 4f


##### . (apply)
A function of arity N applied to a list x of size N is executed with the . verb as .[f;x] or f . x.

In [46]:
{[a;b;c;d] a+2*b+3*c-d} . (4;5;3;7)

-10


In [47]:
// monadic
show {x*x} . enlist 5;
show {x*x}@5;

25
25


applied to lists, the verb . indexes a list at depth

In [48]:
L:((`a;4);(`c;5);(`d;7;(8;6;9f)))
L . 2 2 2

9f


In [49]:
show L;
// get a row in the list by
show L[1];
// get the column in a nested list by
L . (::;1)

(`a;4)
(`c;5)
(`d;7;(8;6;9f))
`c
5


4 5 7


In [50]:
// equvilant notaition
L[::;1]

4 5 7


The verb . can also be applied to dictionaries

In [51]:
d:`a`b`c!((4f;5f;6f);3;`t);
d . (`a;1)

5f


In [52]:
d . (`a;::)

4 5 6f


 This .[L;I;f] applies function f to Ith index of L and returns it, keeping the list unchanged.
While .[`L;I;f] modifies the existing list (by reference)

In [53]:
L:((`a;4);(`c;5);(`d;7;(8;6;9f)));
show .[L;2 2;log];
L

(`a;4)
(`c;5)
(`d;7;2.079442 1.791759 2.197225)


(`a;4)
(`c;5)
(`d;7;(8;6;9f))


In [54]:
show .[`L;2 2;log];
L

`L


(`a;4)
(`c;5)
(`d;7;2.079442 1.791759 2.197225)


### 3.4 Protected Evaluation
The concept of try catch, or protected evaluation, is inplimented with the samew @ and . verbs. The trap function has the syntax @[f;arg;err] where f is a monadic function, and arg its argument. For multivariate funcitons, use .[f;args;err] where args are a list of arguments.  

In [55]:
f:{@[sqrt;x;`err]}

In [56]:
f[9f]

3f


In [57]:
f[`a]

`err


In [58]:
mult:{.[*;(x;y);`err]};
show mult[4;5];
mult[4;"toto"]

20


`err


In [59]:
// customize error handeler
mult:{.[*;(x;y);{"Error is: ",x}]};
mult[4;"toto"]

"Error is: type"


### 3.5 Vector Operations
In this section we will present a few more native functions which cover basic time series operations. We split the functions into two classes: aggregators.

#### 3.5.1 Aggregators
Simple Aggregators:
   - min
   - max
   - avg
   - med
   - var
   - dev
   - svar
   - sdev

In [60]:
L:3 1 4 1 6 7 4 3 8 9 1 2f;
min L

1f


In [61]:
max L

9f


In [62]:
avg L

4.083333


In [63]:
med L

3.5


In [64]:
var L

7.243056


In [65]:
dev L

2.691293


In [66]:
svar L

7.901515


In [67]:
sdev L

2.810963


Weighted Aggregators

Weight aggregators